Imports:

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer

In [14]:
def distance_into_probability(x, w, b):
    ''' Функция возвращает по найденному уравнению разделяющей плоскости вероятность объекта быть идеальным
    x - это признаки данного объекта
    b - вектор коэффициентов в уравнении плоскости '''
    lin_comb = b
    for i in range(len(x)):
        lin_comb += x[i] * w[i]
    return 1 / (1 + np.exp(-lin_comb))

Делим исходные данные на признаки и ответы, а также разделяем на данные, на которых обучаем и на которых тестируем

In [15]:
df = pd.read_csv('../assets/train.csv')

heads = df.keys()[:-1]

X = df[heads]
y = df['result']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [16]:
scaler = StandardScaler().fit(x_train) 
standardized_X = scaler.transform(x_train)

x_train = standardized_X

In [17]:
scaler = StandardScaler().fit(x_test) 
standardized_X = scaler.transform(x_test)

x_test = standardized_X

In [18]:
model = svm.SVC(kernel='linear', C = 1.0, probability=True)

model.fit(x_train, y_train)
w_coeffs = list(model.coef_)[0]
b = model.intercept_

print(w_coeffs)

print(model.intercept_)


[ 0.80638135  0.79093928  0.95710857 -1.28477844 -0.86257068]
[-0.23120921]


In [19]:
line = list(x_test[10])

In [20]:
line, w_coeffs, b[0]

([-1.6997490121370986,
  -0.867187718978134,
  -0.6106412918684773,
  -0.0944998287144107,
  1.0508421319210817],
 array([ 0.80638135,  0.79093928,  0.95710857, -1.28477844, -0.86257068]),
 -0.23120920751315904)

In [21]:
distance_into_probability(line, w_coeffs, b[0]) * 100

2.5155234669017235

In [22]:
des = model.decision_function([x_test[10]])
1 / (1 + np.exp(-des[0]))

0.025155234669017257

Экспортирование модели в папку `/models`

In [23]:
import joblib
joblib.dump(model, '../models/svc.pkl')

['../models/svc.pkl']